# Token classifier model 

In this notebook we will load and use a token classifier language model that assigns labels to some tokens in a text. Learn more about the Question Answering task <a href="https://huggingface.co/tasks/token-classification" target="_blank" rel="noopener">here</a>.

We will be running SQL queries using <a href="https://jupysql.ploomber.io/en/latest/quick-start.html" target="_blank" rel="noopener"> JupySQL</a> SQL Magic.

## Prerequisites

Prior to using this notebook one needs to complete the follow steps:
1. [Configure the sandbox](../sendbox_config.ipynb).
2. [Initialize the Transformer Extension](te_init.ipynb).

## Set up

In [1]:
#TODO: start using the secret store.

from collections import UserDict

class Secrets(UserDict):
    """This class mimics the Secret Store we will start using soon."""

    def save(self, key: str, value: str) -> "Secrets":
        self[key] = value
        return self

# For now just hardcode the configuration.
sb_config = Secrets({
    'EXTERNAL_HOST_NAME': '192.168.124.93',
    'HOST_PORT': '8888',
    'USER': 'sys',
    'PASSWORD': 'exasol',
    'BUCKETFS_PORT': '6666',
    'BUCKETFS_USER': 'w',
    'BUCKETFS_PASSWORD': 'write',
    'BUCKETFS_USE_HTTPS': 'False',
    'BUCKETFS_SERVICE': 'bfsdefault',
    'BUCKETFS_BUCKET': 'default',
    'SCRIPT_LANGUAGE_NAME': 'PYTHON3_60',
    'UDF_FLAVOR': 'python3-ds-EXASOL-6.0.0',
    'UDF_RELEASE': '20190116',
    'UDF_CLIENT': 'exaudfclient_py3',
    'SCHEMA': 'IDA',
    'TE_TOKEN': '',
    'TE_TOKEN_CONN': '',
    'TE_BFS_CONN': 'MyBFSConn',
    'TE_BFS_DIR': 'my_storage',
    'TE_MODELS_BFS_DIR': 'models',
    'TE_MODELS_CACHE_DIR': 'models_cache'
})

EXTERNAL_HOST = f"{sb_config.get('EXTERNAL_HOST_NAME')}:{sb_config.get('HOST_PORT')}"
WEBSOCKET_URL = f"exa+websocket://{sb_config.get('USER')}:{sb_config.get('PASSWORD')}" \
    f"@{EXTERNAL_HOST}/{sb_config.get('SCHEMA')}?SSLCertificate=SSL_VERIFY_NONE"

Let's bring up JupySQL and connect to the database via SQLAlchemy. Please refer to the documentation in the <a href="https://github.com/exasol/sqlalchemy-exasol" target="_blank" rel="noopener">sqlalchemy-exasol</a> for details on how to connect to the database using Exasol SQLAlchemy driver.

In [3]:
from sqlalchemy import create_engine

engine = create_engine(WEBSOCKET_URL)

%load_ext sql
%sql engine

## Get language model

To demonstrate the token classification task we will use an [English Named Entity Recognition model](https://huggingface.co/sschet/biomedical-ner-all), trained on Maccrobat to recognize the bio-medical entities (107 entities) from a given text corpus (case reports etc.).

We need to load the model from the Huggingface hub into the BucketFS. This could potentially be a long process. Unfortunately we cannot tell exactly when it has finished. Notebook's hourglass may not be a reliable indicator. BucketFS will still be doing some work when the call issued by the notebook returns. Please wait for few moments after that, before querying the model.

In [4]:
# This is the name of the model at the Huggingface Hub
MODEL_NAME = 'sschet/biomedical-ner-all'

In [5]:
%run ./model_retrieval.ipynb
load_huggingface_model(MODEL_NAME, sb_config, method='udf')

## Use language model

In [6]:
# We will display all model output
%config SqlMagic.displaylimit = 0

In [7]:
MY_TEXT = """
A 63-year-old woman with no known cardiac history presented with a sudden onset of dyspnea requiring
intubation and ventilatory support out of hospital. She denied preceding symptoms of chest discomfort,
palpitations, syncope or infection. The patient was afebrile and normotensive, with a sinus tachycardia
of 140 beats/min.
"""

# Make sure our texts can be used in an SQL statement.
MY_TEXT = MY_TEXT.replace("'", "''")

In [ ]:
%%sql
WITH MODEL_OUTPUT AS
(
    SELECT TE_TOKEN_CLASSIFICATION_UDF(
        NULL,
        '{{sb_config.get("TE_BFS_CONN")}}',
        '{{sb_config.get("TE_TOKEN_CONN")}}',
        '{{sb_config.get("TE_MODELS_BFS_DIR")}}',
        '{{MODEL_NAME}}',
        '{{MY_TEXT}}',
        NULL
    )
)
SELECT start_pos, end_pos, word, entity, error_message FROM MODEL_OUTPUT ORDER BY start_pos, end_pos